In [3]:
import os
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import dask.dataframe as dd

In [4]:
# Load the mutation dataset
mutations_path = "data/mutations_dataset.csv"
mutations_df = pd.read_csv(mutations_path)

# Create a mutation count matrix
mutation_matrix = (
    mutations_df
    .groupby(['Yeast_ID', 'Protein_ID'])['Position']  # Group by Yeast_ID and Protein_ID
    .count()  # Count the number of mutations for each (Yeast_ID, Protein_ID)
    .unstack(fill_value=0)  # Pivot the table to get a matrix
)

# Load the CNV dataset
cnv_path = "data/genesMatrix_CopyNumber.tab"
cnv_df = pd.read_csv(cnv_path, sep="\t", index_col=0).T  # Transpose the CNV data
cnv_df.index.name = "Yeast_ID"
cnv_df.reset_index(inplace=True)

# Extract the real protein IDs in the CNV dataset
pattern_case1 = r'Y[A-Z]{2}\d{3}[A-Z]'  # Case 1 pattern
pattern_case2 = r'Y[A-Z]{2}\d{3}[A-Z]\.[A-Z]'  # Case 2 pattern
def extract_real_name(name):
    import re
    match_case2 = re.search(pattern_case2, name)
    if match_case2:
        return match_case2.group().replace('.', '-')
    match_case1 = re.search(pattern_case1, name)
    if match_case1:
        return match_case1.group()
    return None
cnv_df['Yeast_ID'] = cnv_df['Yeast_ID'].apply(extract_real_name)
cnv_df = cnv_df.dropna(subset=['Yeast_ID']).reset_index(drop=True)  # Remove invalid rows

# Transpose CNV data back after processing Standard_name
cnv_matrix = cnv_df.set_index('Yeast_ID').T  # Transpose for Yeast_ID rows
cnv_matrix.index.name = 'Yeast_ID'

# Merge mutation count matrix and CNV matrix
final_matrix = mutation_matrix.merge(
    cnv_matrix,
    left_index=True,
    right_index=True,
    how='left',
    suffixes=('', '_CNV')  # Append '_CNV' to CNV columns
)

# Fill missing values in CNV columns with median
cnv_columns = [col for col in final_matrix.columns if col.startswith("CNV_")]
final_matrix[cnv_columns] = final_matrix[cnv_columns].fillna(final_matrix[cnv_columns].median())

# Save the final matrix to a CSV file
output_path = "X_matrix_proteins_and_cnvs_count.csv"
final_matrix.to_csv(output_path, index=True)

print(f"Final matrix with mutation counts saved at {output_path}. Dimensions: {final_matrix.shape}")

Final matrix with mutation counts saved at X_matrix_proteins_and_cnvs_count.csv. Dimensions: (1011, 12542)


In [6]:
import pandas as pd

# Charger les matrices
X_path = "X_matrix_proteins_and_cnvs_count.csv"  # Chemin vers la matrice X
Y_path = "data/Y_matrix.csv"  # Chemin vers la matrice Y

X = pd.read_csv(X_path)
Y = pd.read_csv(Y_path)

# Vérifier la correspondance des Yeast_ID
X_ids = set(X['Yeast_ID'])
Y_ids = set(Y['Yeast_ID'])

# Identifiez les IDs manquants
missing_in_X = Y_ids - X_ids
missing_in_Y = X_ids - Y_ids

if missing_in_X:
    print(f"IDs présents dans Y mais absents dans X : {missing_in_X}")
else:
    print("Tous les IDs de Y sont présents dans X.")

if missing_in_Y:
    print(f"IDs présents dans X mais absents dans Y : {missing_in_Y}")
else:
    print("Tous les IDs de X sont présents dans Y.")

# Aligner les matrices sur les Yeast_ID communs
common_ids = X_ids & Y_ids

# Filtrer X et Y pour ne garder que les IDs communs
X_aligned = X[X['Yeast_ID'].isin(common_ids)].sort_values(by="Yeast_ID")
Y_aligned = Y[Y['Yeast_ID'].isin(common_ids)].sort_values(by="Yeast_ID")

# Vérifier que l'ordre des IDs correspond
if list(X_aligned['Yeast_ID']) == list(Y_aligned['Yeast_ID']):
    print("Les IDs sont alignés entre X et Y.")
else:
    print("Les IDs ne sont pas dans le même ordre, veuillez vérifier.")

# Sauvegarder les matrices alignées si besoin
X_aligned.to_csv("X_matrix_aligned.csv", index=False)
Y_aligned.to_csv("Y_matrix_aligned.csv", index=False)

Tous les IDs de Y sont présents dans X.
IDs présents dans X mais absents dans Y : {'AVH', 'SACE_YBP', 'CLS', 'SACE_YBL', 'SACE_YCL', 'SACE_YBS', 'CHP', 'SACE_YDO', 'ARI', 'SACE_YBG', 'CGE', 'CKN', 'SACE_YDC', 'BFL', 'SACE_YCT', 'CLD', 'SACE_GAS', 'BTG', 'SACE_YCC', 'CHI', 'BKS', 'BEF', 'ABH', 'AIG', 'SACE_YBR', 'CLI', 'ABK', 'CHL', 'SACE_YBT', 'SACE_YBQ', 'SACE_YAC', 'CII', 'AGB', 'SACE_YDK', 'AII', 'SACE_YCI', 'SACE_YCB', 'AIR', 'SACE_MAB', 'CLL', 'BHK', 'BGA', 'CGG', 'CHG', 'BRA', 'BHF', 'BRG', 'SACE_YBO', 'SACE_YAY', 'CGV', 'AHQ', 'SACE_YBW', 'CGR', 'SACE_YDL', 'SACE_YAQ', 'BQT', 'AER', 'ARK', 'CGQ', 'CGT', 'AMS', 'CGL', 'SACE_YCF', 'SACE_YBU', 'SACE_YBB', 'AHF', 'SACE_YDB', 'CHD', 'BKR', 'SACE_YCQ', 'SACE_YCM', 'BDC', 'BRE', 'CHN', 'SACE_YAT', 'BKD', 'CQB', 'ABG', 'ALV', 'SACE_YCG', 'SACE_YAN', 'CFN', 'SACE_YCY', 'APV', 'CGS', 'SACE_GAL', 'AIK', 'CKE', 'BFA', 'CGP', 'SACE_YBZ', 'CHM', 'SACE_MAA', 'SACE_YAX', 'CLT', 'SACE_YBE', 'SACE_YCP', 'ASL', 'CKA', 'AFA', 'BRF', 'SACE_YBF', 'BI

In [7]:
import pandas as pd

# Charger les matrices alignées
X_path =  "X_matrix_aligned.csv"  # Chemin vers la matrice X filtrée
Y_path = "data/Y_matrix.csv"  # Chemin vers la matrice Y

X = pd.read_csv(X_path)
Y = pd.read_csv(Y_path)

# Filtrer Y pour ne garder que les IDs communs
common_ids = set(X['Yeast_ID']) & set(Y['Yeast_ID'])
Y_filtered = Y[Y['Yeast_ID'].isin(common_ids)].sort_values(by="Yeast_ID")

# Vérifier que les IDs sont bien alignés
if list(X['Yeast_ID']) != list(Y_filtered['Yeast_ID']):
    raise ValueError("Les Yeast_ID ne sont pas alignés entre X et Y. Vérifiez vos données.")

# Mélanger les indices aléatoirement tout en maintenant l'alignement
shuffled_indices = X.sample(frac=1, random_state=42).index

X_shuffled = X.loc[shuffled_indices].reset_index(drop=True)
Y_shuffled = Y_filtered.loc[shuffled_indices].reset_index(drop=True)

# Sauvegarder les nouvelles matrices mélangées
X_shuffled.to_csv("X_matrix_proteins_nb.csv", index=False)
Y_shuffled.to_csv("Y_matrix_proteins_nb.csv", index=False)

print(f"Les matrices X et Y ont été mélangées et sauvegardées avec succès.")
print(f"Dimensions de X_shuffled : {X_shuffled.shape}")
print(f"Dimensions de Y_shuffled : {Y_shuffled.shape}")

Les matrices X et Y ont été mélangées et sauvegardées avec succès.
Dimensions de X_shuffled : (792, 12543)
Dimensions de Y_shuffled : (792, 2)


In [8]:
# Sauvegarder les nouvelles matrices mélangées
X_shuffled.to_pickle("X_matrix_proteins_nb.pkl")
Y_shuffled.to_pickle("Y_matrix_proteins_nb.pkl")